In [1]:
import pandas as pd

# read 5guys_flight_data_preprocessed.csv into a dataframe
df = pd.read_csv('data/5guys_flight_data_preprocessed.csv')

In [5]:
# values of DEP_DEL15
print(df['DEP_DEL15'].value_counts())

DEP_DEL15
-0.482720    4404105
 2.071596    1026237
Name: count, dtype: int64


# Train

In [4]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler

# Select columns engineered with feature importance from random forest
features = ['MONTH','SEGMENT_NUMBER','PRCP','CARRIER_NAME','CRS_DEP_TIME']

# target feature
target = 'DEP_DEL15'

# Split the dataset into training and testing sets
X = df[features]
y = df[target]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Perform undersampling on the training data only
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)


ValueError: Unknown label type: 'continuous'

In [2]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
import matplotlib.pyplot as plt
import seaborn as sns

# Select columns engineered with feature importance from random forest
features = ['MONTH','SEGMENT_NUMBER','PRCP','CARRIER_NAME','CRS_DEP_TIME']

# target feature
target = 'DEP_DEL15'

# Split the dataset into training and testing sets
X = df[features]
y = df[target]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)

# Perform undersampling on the training data only
undersampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train, y_train)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# use GridSearchCV to find the best hyperparameters for KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9],
              'weights': ['uniform', 'distance'],
              'p': [1, 2]}

# Create the KNN classifier
model = KNeighborsClassifier()

# Create the GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='f1')

# Fit the GridSearchCV object to the training data
grid_search.fit(X_train_scaled, y_train_resampled)

# Get the best hyperparameters
best_params = grid_search.best_params_
print('Best Hyperparameters:', best_params)

# Train the KNN classifier with the best hyperparameters
best_knn = KNeighborsClassifier(**best_params)
best_knn.fit(X_train_scaled, y_train_resampled)

# Predict on the test data using the best classifier
y_pred_best = best_knn.predict(X_test_scaled)
y_pred_proba_best = best_knn.predict_proba(X_test_scaled)[:, 1]

# Perform cross-validation for scores
# scores = cross_val_score(model, X_train_scaled, y_train_resampled, cv=cv, scoring='f1')

# Print the mean of the cross validation scores
# print(f'Scores for each fold: {scores}')
# print(f'Mean f1 score: {scores.mean():.3f}')

# Train classifier
# model.fit(X_train_scaled, y_train_resampled)

# Predict on test data
# y_pred = model.predict(X_test_scaled)
# y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# Calculate the ROC curve points
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_best)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 8))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

# Display metrics
# Evaluate the model on the original test set
accuracy = accuracy_score(y_test, y_pred_best)
confusion = confusion_matrix(y_test, y_pred_best)
report = classification_report(y_test, y_pred_best)

print(f'Accuracy: {accuracy}')
print('Confusion Matrix:')
print(confusion)
print('Classification Report:')
print(report)

# Confusion matrix
import seaborn as sns
plt.figure(figsize=(6, 6))
sns.heatmap(confusion, annot=True, cmap='rocket', fmt='g', cbar=True)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (On Original Test Data)')
plt.show()

ValueError: Unknown label type: 'continuous'